In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import utils
import models

In [2]:
model = models.build_model("resnet152")
base_model = model.layers[1]

In [6]:
len(base_model.layers)

515

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
model = models.resnet152_v2()

In [3]:
MAIN_PATH = "/mnt/homeGPU/fcastro/MINDAT_AVAN/fran/"

#data_dir = "../reduced_data/"
data_dir = MAIN_PATH + "data/"
batch_size = 32
img_height = 224
img_width = 224

load = False
weights_dir = MAIN_PATH + "weights/"
load_weights_file = "resnet152_ft10_DA1.h5"
save_weights_file = "resnet152_baseline_DA2.h5"
load_weights_path = weights_dir + load_weights_file
save_weights_path = weights_dir + save_weights_file

early_stop = False
patience = 10
epochs = 200
learning_rate = 0.0001
last_ft_layers = 2

DA_fn = models.DA2()

In [4]:
train_data, train_labels, test_data, test_labels, test_names = utils.load_data(data_dir, norm=False)
train_data = tf.keras.applications.resnet.preprocess_input(train_data)
test_data = tf.keras.applications.resnet.preprocess_input(test_data)
train_labels = tf.one_hot(train_labels, 29).numpy()
test_labels = tf.one_hot(test_labels, 29).numpy()

idx = np.arange(len(train_data))
train_idx, val_idx = train_test_split(idx,test_size=0.1, random_state=0)
val_data = train_data[val_idx]
val_labels = train_labels[val_idx]
train_data = train_data[train_idx]
train_labels = train_labels[train_idx]

In [5]:
def prep_fn(img):
    img = tf.expand_dims(img, axis=0)
    new_img = DA_fn(img)[0]
    return new_img

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=prep_fn)
train_generator.fit(train_data)
val_generator = tf.keras.preprocessing.image.ImageDataGenerator()

In [6]:
#model = models.resnet50_baseline()
model = models.resnet152_baseline()

if (load):
        model.load_weights(load_weights_path)
        
model.trainable = True
num_layers = len(model.layers)
for layer in model.layers[:num_layers-last_ft_layers]:
        layer.trainable = False

In [7]:
loss = tf.keras.losses.CategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
metrics = [tf.keras.metrics.categorical_accuracy]
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=0, mode='auto', restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(save_weights_path, save_best_only=True, save_weights_only=True)

if early_stop:
    callbacks = [model_checkpoint, early_stopping]
else:
    callbacks = [model_checkpoint]


In [8]:
"""
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

train_it = train_generator.flow(train_data, train_labels, batch_size)
val_it = val_generator.flow(val_data, val_labels, 8)

#hist = model.fit(train_data, train_labels, epochs=30, callbacks=callbacks, validation_data=(test_data, test_labels), batch_size=128)
hist = model.fit(train_it, validation_data=val_it, epochs=1)
"""

'\nmodel.compile(optimizer=optimizer, loss=loss, metrics=metrics)\n\ntrain_it = train_generator.flow(train_data, train_labels, batch_size)\nval_it = val_generator.flow(val_data, val_labels, 8)\n\n#hist = model.fit(train_data, train_labels, epochs=30, callbacks=callbacks, validation_data=(test_data, test_labels), batch_size=128)\nhist = model.fit(train_it, validation_data=val_it, epochs=1)\n'

In [9]:
train_it = train_generator.flow(train_data, train_labels, batch_size)
val_it = val_generator.flow(val_data, val_labels, 8)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
hist = model.fit(train_it, validation_data=val_it, epochs=epochs, callbacks=callbacks)

Epoch 1/200


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-9-55e07724e0c3>:5) ]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-9-55e07724e0c3>:5) ]]
	 [[IteratorGetNext/_3]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_29651]

Function call stack:
train_function -> train_function


In [ ]:
preds = model.predict(test_data, batch_size)
acc = accuracy_score(np.argmax(test_labels, axis=-1), np.argmax(preds, axis=-1))
f1 = f1_score(np.argmax(test_labels, axis=-1), np.argmax(preds, axis=-1),  average="micro")
print(acc)
print(f1)

0.10444993819530285
0.10444993819530286
